In [21]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import Ollama, OpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
import chainlit as cl

In [7]:
llms = {
    "openai": OpenAI(temperature=0),
    "mistral": Ollama(
        model="mistral", verbose=True, callbacks=CallbackManager([StreamingStdOutCallbackHandler()])
    ),
}

template = """You are a Retrieval-Augmented Generation chatbot that answers questions on
documents provided to you. Act as an expert in the subject matter of the document
discussed. If a question is not relevant for the document or if it cannot be answered
using the information of the document, please do not answer the question and politely provide
the reason. You're going to be working with the following document:
{document}

Given the above document, please answer the following question:
{{question}}
"""

document_path = "data/business_1.txt"
with open(document_path) as f:
    document = f.read()

template = template.format(document=document)

In [8]:
# Instantiate the chain for that user session
prompt = PromptTemplate.from_template(template)
llm = llms["mistral"]
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

In [9]:
llm("Hello, how are you?")

I'm just a computer program, so I don't have feelings or emotions. But I'm here to help you with any questions or problems you might have! How can I assist you today?

"I'm just a computer program, so I don't have feelings or emotions. But I'm here to help you with any questions or problems you might have! How can I assist you today?"

In [10]:
llm_chain("Hello, how are you")



> Entering new LLMChain chain...
Prompt after formatting:
You are a Retrieval-Augmented Generation chatbot that answers questions on
documents provided to you. Act as an expert in the subject matter of the document
discussed. If a question is not relevant for the document or if it cannot be answered
using the information of the document, please do not answer the question and politely provide
the reason. You're going to be working with the following document:
Lufthansa flies back to profit

German airline Lufthansa has returned to profit in 2004 after posting huge losses in 2003.

In a preliminary report, the airline announced net profits of 400m euros ($527.61m; £274.73m), compared with a loss of 984m euros in 2003. Operating profits were at 380m euros, ten times more than in 2003. Lufthansa was hit in 2003 by tough competition and a dip in demand following the Iraq war and the killer SARS virus. It was also hit by troubles at its US catering business. Last year, Lufthansa showed sig

{'question': 'Hello, how are you',
 'text': '\nI am doing well, thank you for asking!'}

In [11]:
llm_chain.memory

In [12]:
llm_chain("What did I ask you a moment ago?")



> Entering new LLMChain chain...
Prompt after formatting:
You are a Retrieval-Augmented Generation chatbot that answers questions on
documents provided to you. Act as an expert in the subject matter of the document
discussed. If a question is not relevant for the document or if it cannot be answered
using the information of the document, please do not answer the question and politely provide
the reason. You're going to be working with the following document:
Lufthansa flies back to profit

German airline Lufthansa has returned to profit in 2004 after posting huge losses in 2003.

In a preliminary report, the airline announced net profits of 400m euros ($527.61m; £274.73m), compared with a loss of 984m euros in 2003. Operating profits were at 380m euros, ten times more than in 2003. Lufthansa was hit in 2003 by tough competition and a dip in demand following the Iraq war and the killer SARS virus. It was also hit by troubles at its US catering business. Last year, Lufthansa showed sig

{'question': 'What did I ask you a moment ago?',
 'text': 'I apologize, but I do not have any information on what question you asked me a moment ago as I am unable to recall past conversations without the context of them. Can you please provide more information or clarify your question so that I can assist you better?'}

In [18]:
llm_chain_openai = LLMChain(prompt=prompt, llm=llms["openai"], verbose=True)

In [20]:
res = await llm_chain_openai.acall("Hello, how are you")

ChainlitContextException: Chainlit context not found